In [1]:
import re
import math
#Function to extract the path from the gcode file and store it in an array
def fileToArray(path):
    filepath = path;
    array = []
    with open(filepath) as fp:
        for line in fp:
            array.append([line])
    return array
#Function to generate tool orientation
def manipulateFile(array):
    #Setting boolean value for first iteration
    firstIteration = False;
    #Setting the storePrevious array
    storePrevious = [1,2,3,4,5]
    #Setting initial values for IJK to set the initial cooridante to (0,0,1), the tool home orientation
    prevI=0.000
    prevJ=0.000
    prevK=1.000
    #Setting the RegEx
    r = re.compile('G1.*X.*Y.*Z.*E.*')
    #Iterating through the entire legnth of the input file
    for i in range (len(array)):
        #Check if the current line of the array matches the RegEx
        newlist = list(filter(r.match, array[i]))
        if (newlist != []):
            #Sperating the string into a comma seperates list
            tempString = ", ".join(newlist).split()
            #Stores the first instance in correct format and sets storePrevious values to t
            if (firstIteration == False):
                storePrevious = tempString
                firstIteration = True
            if (storePrevious[3] == tempString[3]):
                newString = "{} {} {} {} I{} J{} K{} {}\n".format(tempString[0], tempString[1], tempString[2], tempString[3], prevI, prevJ, prevK, tempString[4])
                array[i] = [newString]
            else:
                I = -(float(tempString[2][1:])-float(storePrevious[2][1:]))
                J = -(float(tempString[1][1:])-float(storePrevious[1][1:]))
                K = (float(tempString[3][1:])-float(storePrevious[3][1:]))
                if K < 0:
                    I = -I
                    J = -J
                    K = -K
                V = (I*I + J*J + K*K)**0.5
                if V != 0:
                    I = format((I/V),".3f")
                    J = format((J/V),".3f")
                    K = format((K/V),".3f")
                
                newString = "{} {} {} {} I{} J{} K{} {}\n".format(tempString[0], tempString[1], tempString[2], tempString[3], I, J, K, tempString[4])
                array[i] = [newString]
                
                prevI=I
                prevJ=J
                prevK=K
            storePrevious = tempString
    return array

def arrayToFile(path, data):
    with open(path, "w") as txt_file:
        for line in data:
            txt_file.write(" ".join(line))


In [2]:
theFile = fileToArray('CurvyslThick.gcode')
new_array = manipulateFile(theFile)
arrayToFile("CurvyslThick_IJK.gcode", new_array)